## Student Info
Name: Susanna Mir
Cohort: DS-PT
Instructor: James Irving

## Introduction

>The main goal of this project 

>The data analysed 

>The target 

# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
plt.style.use('ggplot')

import warnings
#warnings.filter.options(‘ignore’)
#plt.style.use(‘seaborn-poster')
#import functions_mod4_proj as  fc_mod4


In [2]:
# # Load the dataset
# dataset = sm.datasets.co2.load().data

# # Convert into DataFrame
# df = pd.DataFrame(dataset)

# # Update to datetime type
# df['date'] = pd.to_datetime(df['index'])

# # Set as index
# df.set_index(df['date'], inplace=True)

# df.drop(['date', 'index'], axis=1, inplace=True)
# df = df.asfreq('W-SAT')

# # The 'MS' string groups the data in buckets by start of the month
# CO2 = df['co2'].resample('MS').mean()

# # The term bfill means that we use the value before filling in missing values
# CO2 = CO2.fillna(CO2.bfill())

# # Plot the time series
# CO2.plot(figsize=(15, 6))
# plt.show()
# print(CO2.head())

In [3]:
data = pd.read_csv('zillow_data.csv')

In [4]:
data.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


### Check Null Values

In [5]:
## Data Clean check - are there any null values?

In [6]:
nulls = data.isna().sum()
nulls[nulls >0]

Metro      1043
1996-04    1039
1996-05    1039
1996-06    1039
1996-07    1039
1996-08    1039
1996-09    1039
1996-10    1039
1996-11    1039
1996-12    1039
1997-01    1039
1997-02    1039
1997-03    1039
1997-04    1039
1997-05    1039
1997-06    1039
1997-07    1038
1997-08    1038
1997-09    1038
1997-10    1038
1997-11    1038
1997-12    1038
1998-01    1036
1998-02    1036
1998-03    1036
1998-04    1036
1998-05    1036
1998-06    1036
1998-07    1036
1998-08    1036
           ... 
2012-01     224
2012-02     224
2012-03     224
2012-04     224
2012-05     224
2012-06     224
2012-07     206
2012-08     206
2012-09     206
2012-10     206
2012-11     206
2012-12     206
2013-01     151
2013-02     151
2013-03     151
2013-04     151
2013-05     151
2013-06     151
2013-07     109
2013-08     109
2013-09     109
2013-10     109
2013-11     109
2013-12     109
2014-01      56
2014-02      56
2014-03      56
2014-04      56
2014-05      56
2014-06      56
Length: 220, dtype: int6

### Examine Data Types

In [23]:
#fs.quick_refs.ts_pandas_freq_aliases()

In [33]:
def make_datetimeindex(df, col_to_make_index='Month', drop=False, verbose=True):
    
        col_to_make_index = 'Month'
        df[col_to_make_index] = pd.to_datetime(df[col_to_make_index].errors='coerce')
        df=df.set_index('Month',drop=false)
        df.index

        if verbose:
            display(df.index)
        return df


SyntaxError: keyword can't be an expression (<ipython-input-33-d138706276b1>, line 4)

In [36]:
df2 = data.groupby('RegionName').resample('MS')
df2

df2.isna().sum()

df2['Metro'].fillna('Missing', inplace=True)
df2.isna().sum()


KeyError: 'RegionName'

In [7]:
data.dtypes

RegionID        int64
RegionName      int64
City           object
State          object
Metro          object
CountyName     object
SizeRank        int64
1996-04       float64
1996-05       float64
1996-06       float64
1996-07       float64
1996-08       float64
1996-09       float64
1996-10       float64
1996-11       float64
1996-12       float64
1997-01       float64
1997-02       float64
1997-03       float64
1997-04       float64
1997-05       float64
1997-06       float64
1997-07       float64
1997-08       float64
1997-09       float64
1997-10       float64
1997-11       float64
1997-12       float64
1998-01       float64
1998-02       float64
               ...   
2015-11         int64
2015-12         int64
2016-01         int64
2016-02         int64
2016-03         int64
2016-04         int64
2016-05         int64
2016-06         int64
2016-07         int64
2016-08         int64
2016-09         int64
2016-10         int64
2016-11         int64
2016-12         int64
2017-01   

In [8]:
data.describe()

,RegionID,RegionName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
count,14723.000000,14723.000000,14723.000000,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,...,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04
mean,81075.010052,48222.348706,7362.000000,1.182991e+05,1.184190e+05,1.185374e+05,1.186531e+05,1.187803e+05,1.189275e+05,1.191205e+05,...,2.733354e+05,2.748658e+05,2.764646e+05,2.780332e+05,2.795209e+05,2.810953e+05,2.826571e+05,2.843687e+05,2.865114e+05,2.880399e+05
std,31934.118525,29359.325439,4250.308342,8.600251e+04,8.615567e+04,8.630923e+04,8.646795e+04,8.665094e+04,8.687208e+04,8.715185e+04,...,3.603984e+05,3.614678e+05,3.627563e+05,3.644610e+05,3.656003e+05,3.670454e+05,3.695727e+05,3.717739e+05,3.724612e+05,3.720544e+05
min,58196.000000,1001.000000,1.000000,1.130000e+04,1.150000e+04,1.160000e+04,1.180000e+04,1.180000e+04,1.200000e+04,1.210000e+04,...,1.440000e+04,1.450000e+04,1.470000e+04,1.480000e+04,1.450000e+04,1.430000e+04,1.410000e+04,1.390000e+04,1.380000e+04,1.380000e+04
25%,67174.500000,22101.500000,3681.500000,6.880000e+04,6.890000e+04,6.910000e+04,6.920000e+04,6.937500e+04,6.950000e+04,6.960000e+04,...,1.269000e+05,1.275000e+05,1.282000e+05,1.287000e+05,1.292500e+05,1.299000e+05,1.306000e+05,1.310500e+05,1.319500e+05,1.324000e+05
50%,78007.000000,46106.000000,7362.000000,9.950000e+04,9.950000e+04,9.970000e+04,9.970000e+04,9.980000e+04,9.990000e+04,9.995000e+04,...,1.884000e+05,1.896000e+05,1.905000e+05,1.914000e+05,1.925000e+05,1.934000e+05,1.941000e+05,1.950000e+05,1.967000e+05,1.981000e+05
75%,90920.500000,75205.500000,11042.500000,1.432000e+05,1.433000e+05,1.432250e+05,1.432250e+05,1.435000e+05,1.437000e+05,1.439000e+05,...,3.050000e+05,3.066500e+05,3.085000e+05,3.098000e+05,3.117000e+05,3.134000e+05,3.151000e+05,3.168500e+05,3.188500e+05,3.211000e+05
max,753844.000000,99901.000000,14723.000000,3.676700e+06,3.704200e+06,3.729600e+06,3.754600e+06,3.781800e+06,3.813500e+06,3.849600e+06,...,1.888990e+07,1.870350e+07,1.860530e+07,1.856940e+07,1.842880e+07,1.830710e+07,1.836590e+07,1.853040e+07,1.833770e+07,1.789490e+07


In [9]:
data.shape

(14723, 272)

In [10]:
data=data.dropna()

### Check for Outliers

In [11]:
# # Create a box plot of each column and check for significant outliers
# plt.figure(figsize=(20, 10))
# plt.boxplot([data[col] for col in data.columns])
# plt.title("Box plot of all columns in dataset")
# plt.xticks(range(len(data.columns.values)), data.columns.values)
# plt.show()

### Check for Unique Values

In [12]:
# checking how many unique values we have on each column, besides date/time data
data.iloc[:,0:7].nunique()

RegionID      12895
RegionName    12895
City           6591
State            50
Metro           642
CountyName      917
SizeRank      12895
dtype: int64

In [13]:
# rename column that refers to zipcodes
data.rename({'RegionName': 'Zipcode'}, axis='columns', inplace=True)
data.head()

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


# Step 2: Data Preprocessing

In [14]:
# filter data by zipcode (group by) ?????????? --- ********** James how to seperate zipcode data
df_chicago = data.loc[data['City'] == 'Chicago'].copy()

In [15]:
df_chicago.head()

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
7,84640,60640,Chicago,IL,Chicago,Cook,8,216500.0,216700.0,216900.0,...,798000,787100,776100,774900,777900,777900,778500,780500,782800,782800
16,84646,60647,Chicago,IL,Chicago,Cook,17,122700.0,122800.0,122800.0,...,459600,461400,464300,466500,467900,470600,474500,475100,472600,470200
19,84620,60618,Chicago,IL,Chicago,Cook,20,142600.0,143100.0,143400.0,...,462700,461200,459900,459200,458700,457900,457400,459000,462500,464300


In [16]:
# Melt data and make it into time series

def melt_data(df):
    ''' 
    Takes a dataframe with datetime data that is in wide format and melts it into long format; 
    Tranforms data into datetime object with time as index.
                    df          Dataframe to transform
    '''
    
    melted = pd.melt(df, id_vars=['RegionID', 'Zipcode', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted
    #return melted.groupby('time').aggregate({'value':'mean'})


In [17]:
# apply function to our data:
melted_df_chicago = melt_data(df_chicago)

In [18]:
melted_df_chicago.head()

,RegionID,Zipcode,City,State,Metro,CountyName,SizeRank,time,value
0,84654,60657,Chicago,IL,Chicago,Cook,1,1996-04-01,334200.0
1,84616,60614,Chicago,IL,Chicago,Cook,4,1996-04-01,498100.0
2,84640,60640,Chicago,IL,Chicago,Cook,8,1996-04-01,216500.0
3,84646,60647,Chicago,IL,Chicago,Cook,17,1996-04-01,122700.0
4,84620,60618,Chicago,IL,Chicago,Cook,20,1996-04-01,142600.0


# Step 3: EDA and Visualization

In [ ]:
#Ts = df2[‘MeanValue’].loc[21207]
def plot_ts(df2, col='MeanValue', zipcodes=[21201]):

Ax = ts.plot(label='Raw Price')
fig, ax = plt.subplots()

for zc in zipcodes:
    ts = df2[col].loc[zc]

    ts.plot(label=str(zc), ax=ax)

    #Max_ = ts.loc[‘2004’:’2010’].idxmax()
    Crash = '01-2009'
    #Min_ = tx.loc[crsh:].idxmin()

    #Ax.axvline(max_, label=’Max Price’, color =’green’, ls=’:’)
    Ax.axvline(crash, label =’Housing Index Drops’, color=’red’, ls=’:’)
    #Ax.axvline(min_, label =Min Price Post-Crash, color=’black’, ls=’:’)

    Ax.legend()

    Return fig,ax

Fig, ax = plot_ts(df2, zipcodes=[21201,21207,1267,60625, 90025]


In [ ]:
data = df2.loc[pd.IndexSlic[:,’2014-01-01’:],:].copy()
data.head()

In [ ]:
data.reset_index(inplace=True,level=0,drop=True)
data.head()

In [ ]:
#Ts=df[‘MeanValue’].loc[‘2010:]
ts =data.loc[df['RegionName']==1267]["MeanValue".rename(1267)
ts.head()
#Ts=ts[‘MeanValue’]
#TS=TS.rename[‘1267’]

ts.index

ts = Ts.resample('MS').asfreq()

ts.name

ts.plot()

In [39]:
def plot_acf_pacf(ta, figsize=(10,6),lags=15):
    
    fig,ax = plt.subplots(nrows=2,figsize)
    plot_acf(ts,ax=ax[0],lags=lags)
    plot_pacf(ts,ax=ax[1],lags=lags)
    plt_tight_layout()

    for a in ax:
        a.xaxis.set_major_locator(mpl.ticker.MaxNLocator(min_n_ticks=lags, integer=True))
        a.xaxis.grid()

plot_acf_pacf(ts)


SyntaxError: positional argument follows keyword argument (<ipython-input-39-0e7e7524829f>, line 3)

In [19]:
# font = {'family' : 'normal',
#         'weight' : 'bold',
#         'size'   : 22}

# matplotlib.rc('font', **font)

# # NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [20]:
# Plot the time series
melted_df_chicago.plot(figsize=(12,6), linewidth=2, fontsize=12)
plt.xlabel('Year', fontsize=20)
plt.ylabel('Value', fontsize=12);

ValueError: view limit minimum -73723.95 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x1c2304a620> (for post_execute):


ValueError: view limit minimum -73723.95 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

ValueError: view limit minimum -73723.95 is less than 1 and is an invalid Matplotlib date value. This often happens if you pass a non-datetime value to an axis that has datetime units

If you plotted the time series correctly, you should notice that it is not stationary. So, difference the data to get a stationary time series. Make sure to remove the missing values.

#### White Noise

#### Difference time series

In [ ]:
# Difference the time series
data_diff = melted_df_chicago.diff().dropna()
data_diff

#### Correlations

Use statsmodels to plot the ACF and PACF of this differenced time series.

In [ ]:
# Plot the ACF
from statsmodels.graphics.tsaplots import plot_acf
fig, ax = plt.subplots(figsize=(8,3))
plot_acf(data_diff,ax=ax, lags=8);

In [ ]:
# Plot the PACF
from statsmodels.graphics.tsaplots import plot_pacf
fig, ax = plt.subplots(figsize=(8,3))
plot_pacf(data_diff,ax=ax, lags=8);

In [ ]:
Decomp = seasonal_decompose(ts)
Decomp.plot()

Decomp = seasonal_decompose(ts)
Ts_seas = decomp.seasonal

Ax = ts_seas.plot()
Fig = ax.get_figure()
Fig.set_size_inches(10,4)

#get min and max idx
min_ = ts_seas.idxmin()
max_ = ts_seas.idxmax()
min_2 = ts_seas.loc[max_:].idxmin()

ax.axvline(min_, label=min, c=’orange’)
ax.axvline(max_, c=’orange’, ls=’:’)
ax.axvline(min_2, c=’orange’)

period = min_2 – min_
ax.set_title(f’Season Length = {period}’)


#### Seasonal Decomposition

In [ ]:
decomp = seasonal_decompose(ts)
decomp.plot()

In [ ]:
decomp = seasonal_decompose(ts)
ts_seas = decomp.seasonal

In [44]:
ax = ts_seas.plot()
fig = ax.get_figure()
fig.set_size_inches(10,4)

#get min and max idx
min_ = ts_seas.idxmin()
max_ = ts_seas.idxmax()
min_2 = ts_seas.loc[max_:].idxmin()

ax.axvline(min_, label=min, c='orange')
ax.axvline(max_, c='orange', ls=':')
ax.axvline(min_2, c='orange')

period = min_2 - min_
ax.set_title(f'Season Length = {period}')

NameError: name 'ts_seas' is not defined

# Step 5: ARIMA Modeling

Based on the ACF and PACF, fit an ARMA model with the right orders for AR and MA. Feel free to try different models and compare AIC and BIC values, as well as significance values for the parameter estimates.

In [ ]:
# Import ARMA
from statsmodels.tsa.arima_model import ARMA

# Fit an ARMA(1,0) model
mod_arma = ARMA(data_diff, order=(1,0))
res_arma = mod_arma.fit()

# Print out summary information on the fit
print(res_arma.summary())

In [ ]:
# Fit an ARMA(2,1) model
mod_arma = ARMA(data_diff, order=(2,1))
res_arma = mod_arma.fit()

# Print out summary information on the fit
print(res_arma.summary())

In [ ]:
# Fit an ARMA(2,2) model
mod_arma = ARMA(data_diff, order=(2,2))
res_arma = mod_arma.fit()

# Print out summary information on the fit
print(res_arma.summary())

### Parameter Selection for the ARIMA Time Series Model

The first step towards fitting an ARIMA model is to find the values of ARIMA(p,d,q)(P,D,Q)s that produce the desired output. Selection of these parameters requires domain expertise and time. We shall first generate small ranges of these parameters and use a "grid search" to iteratively explore different combinations of parameters. For each combination of parameters, we fit a new seasonal ARIMA model with the SARIMAX() function from the statsmodels library and assess its overall quality.

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

### AIC (Akaike Information Criterion) as Regularization Measure

The Akaike information criterion (AIC) is an estimator of the relative quality of statistical models for a given set of data. Given a collection of models for the data, AIC estimates the quality of each model, relative to each of the other models. Thus, AIC provides a means for model selection.

A model that fits the data very well while using lots of features will be assigned a larger AIC score than a model that uses fewer features to achieve the same goodness-of-fit. Therefore, we are interested in finding the model that yields the lowest AIC value.

In [ ]:
# Run a grid with pdq and seasonal pdq parameters calculated above and get the best AIC value
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(CO2,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
            print('ARIMA {} x {}12 : AIC Calculated ={}'.format(comb, combs, output.aic))
        except:
            continue

In [ ]:
# Plug the optimal parameter values into a new SARIMAX model
ARIMA_MODEL = sm.tsa.statespace.SARIMAX(data_diff, 
                                        order=(1, 1, 1), 
                                        seasonal_order=(1, 1, 1, 12), 
                                        enforce_stationarity=False, 
                                        enforce_invertibility=False)

# Fit the model and print results
output = ARIMA_MODEL.fit()

print(output.summary().tables[1])

The model returns a lot of information, but we'll focus only on the table of coefficients. The coef column above shows the importance of each feature and how each one impacts the time series patterns. The $P>|z|$ provides the significance of each feature weight.

For our time-series, we see that each weight has a p-value lower or close to 0.05, so it is reasonable to retain all of them in our model.

Next, we shall run model diagnostics to ensure that none of the assumptions made by the model have been violated.

In [ ]:
# Call plot_diagnostics() on the results calculated above 
output.plot_diagnostics(figsize=(15, 18))
plt.show()

The purpose here is to ensure that residuals remain uncorrelated, normally distributed having zero mean. In the absence of these assumptions, we can not move forward and need further tweaking of the model.

Let's check for these assumptions from diagnostics plots.

In the top right plot, we see that the red KDE line follows closely with the N(0,1) line (where N(0,1)) is the standard notation for a normal distribution with mean 0 and standard deviation of 1). This is a good indication that the residuals are normally distributed.

The qq-plot on the bottom left shows that the ordered distribution of residuals (blue dots) follows the linear trend of the samples taken from a standard normal distribution with N(0, 1). Again, this is a strong indication that the residuals are normally distributed.

The residuals over time (top left plot) don't display any obvious seasonality and appear to be white noise. This is confirmed by the autocorrelation (i.e. correlogram) plot on the bottom right, which shows that the time series residuals have low correlation with lagged versions of itself.

These observations lead us to conclude that our model has no correlations and provides a satisfactory fit to help forecast future values.

### Validating the Model

In order to validate the model, we start by comparing predicted values to real values of the time series, which will help us understand the accuracy of our forecasts.

In the cell below:

Get the predictions from 1st January 1998 till 2002 (end of time series)

Get the confidence intervals for all predictions

For get_predictions(), set the dynamic parameter to False to ensure that we produce one-step ahead forecasts, meaning that forecasts at each point are generated using the full history up to that point

In [45]:
mpl.rcParams['figure.figsize']=[12,5]
ts.plot()
plt.figure()

NameError: name 'mpl' is not defined

In [ ]:
for i in range(3):
    ax = ts.diff(i).plot(label=i) 
    plt.legend()

In [ ]:
import itertools
q_range = range(0.10)
d_range = range(1,3)
m_range=(0,6,12)

pdq = list(itertools.product(p_range, d_range, q_range))
PDQM = list(itertools.product(p_range,d_range,q_range,m_range))
PDQM[:10]

In [ ]:

def make_pdq_pdqm(p_range=range(0,10), q_range = range(0,10), 
    d_range=range(1,3), make_seasonal=True, m_values=(0,6,12)):

    import itertools
    ps = range(p_range[0],p_range[1])
    ds = range(d_range[0],d_range[1])
    qs = range(q_range[0],q_range[1])

    params = {}
    params['pdq'] = list(itertools.product(ps,ds,qs))

    return params

    params = make_pdq_pdqm()
    params
    params.keys()
    if make_seasonal:
        params['PDQM'] = list(itertools.product(ps,ds,qs,m_values))
    return params

#Pdq = list(itertools.product(p_range, d_range, q_range))
#PDQM = list(itertools.product(p_range, d_range, q_range, m_range))
#PDQM(:10)

In [ ]:
def get_train_test_split_index(ts, TEST_SIZE=0.1, min_Test_ts=2):
    import math
    idx_split – math.floor(len(ts.index)*(1-TEST_SIZE))

    num_test_ts=len(ts.iloc[idx_split:])
    if num_test_ts < min_test_ts:
        print(f'[!] Warning: unsing TEST_SIZE={TEST_SIZE} produced {num_test_ts} test timestamps')
        print(" - Overriding TEST_SIZE and using min_test_ts instead.")
        idx_split = len(ts)-min_test_ts
    return idx_split

In [ ]:
def train_test_split_ts(ts,test_size=0.2, min_test_ts=2):
    idx_split = get_train_test_split_index(ts, TEST_SIZE=test_size, min_test_ts=min_test_ts)

    ts_train = ts.iloc[:idx_split]
    ts_Test = ts.iloc[idx_split:[

    return ts_train, ts_test

ts_train, ts_test = train_test_split_ts(ts)

len(ts_train), len(ts_test)

In [ ]:
import tqdm
from tqdm import trange


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
def grid_search_sarimax(ts_train,pdq=None,pdqm=None,order_dict=None,verbose=False):

    from tdqm import trange

    if (pdq is None) & (order_dict is not None):
        pdq= order_dict[‘pdq’]
    if (pdqm is None) & (order_dict is not None):
        pdqm= order_dict['PDQM']

    import tqdm
    from tqdm import trange

    start, start_str = get+now(return_dt=True, return_str=True)
    print(f'[i] STARTING GRID SEARCH @ {start_str}:')

    res = [['pdq', 'PDQM', 'IAC']]
        
    for i in trange((len(pdq))):
        comb = pdq[i]
    for combs in pdqm:
        try:
        model = SARIMAX(ts_train, order=comb, seasonal_order=combs, enforce_stationarity=False, enforce_invertibility=False)

        output = model.fit()
        res.append([comb,combs,output.sic])
    except:
    if verbose:
        print(f'[!] Error running ({comb})({combs})')
    continue

    end, end_str = get_now(return_det=True, return_str=True)
    elapsed = end-start
    print(f'[i] GRID SEARCH FINISHED AT {end_str}')
    print(f'\totTotal Time: {elapsed} sec')

    try:
        df_res = fs.list2df(res)
        return df_res
        except:
        print(‘Error converting to df. Retuning as list.’)
    return res


In [ ]:
res = grid_search_sarimax(ts_train,order_dict=params)#pdq,pdqm)

# Step 6: Interpreting Results

ARMA(1,0), ARMA(2,2) and ARMA(2,1) all seem to have decent fits with significant parameters. 
Depending on whether you pick AIC or BIC as a model selection criterion, 
your result may vary. In this situation, you'd generally go for a model with fewer parameters, 
so ARMA(1,0) seems fine. Note that we have a relatively short time series, 
which can lead to a more difficult model selection process.

"\nARMA(1,0), ARMA(2,2) and ARMA(2,1) all seem to have decent fits with significant parameters. \nDepending on whether you pick AIC or BIC as a model selection criterion, \nyour result may vary. In this situation, you'd generally go for a model with fewer parameters, \nso ARMA(1,0) seems fine. Note that we have a relatively short time series, \nwhich can lead to a more difficult model selection process.\n"

In [ ]:
# Get predictions starting from 05-01-1996 and calculate confidence intervals
pred = output.get_prediction(start=pd.to_datetime('1996-05-01'), dynamic=False)
pred_conf = pred.conf_int()

We shall now plot the real and forecasted values of the CO2 time series to assess how well we did:

In [ ]:
# Plot real vs predicted values along with confidence interval

rcParams['figure.figsize'] = 15, 6

# Plot observed values
ax = data_diff['1990':].plot(label='observed')

# Plot predicted values
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='g', alpha=0.5)

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Value')
plt.legend()

plt.show()

The forecasts align with the true values as seen above, with overall increase trend. We shall also check for the accuracy of our forecasts using MSE (Mean Squared Error). This will provide us with the average error of our forecasts. For each predicted value, we compute its distance to the true value and square the result. The results need to be squared so that positive/negative differences do not cancel each other out when we compute the overall mean.

In [ ]:
# Get the real and predicted values
chicago_forecasted = pred.predicted_mean
chicago_truth = data_diff['1996-05-01':]

# Compute the mean square error
mse = ((chicago_forecasted - chicago_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

The MSE of our one-step ahead forecasts yields a value of 0.07, which is very low. An MSE this close to 0 indicates that the estimator is predicting observations of the parameter with perfect accuracy, which would be an ideal scenario but it is not typically possible.

### Dynamic Forecasting

We can achieve a deeper insight into model's predictive power using dynamic forecasts. In this case, we only use information from the time series up to a certain point, and after that, forecasts are generated using values from previous forecasted time points.

Repeat above calculation for predictions post 1998. Use Dynamic forecasting by setting dynamic to True.

In [ ]:
# Get dynamic predictions with confidence intervals as above 
pred_dynamic = output.get_prediction(start=pd.to_datetime('1996-05-01'), dynamic=True, full_results=True)
pred_dynamic_conf = pred_dynamic.conf_int()

Plotting the observed and forecasted values of the time series, we see that the overall forecasts are accurate even when using dynamic forecasts. All forecasted values (red line) match pretty closely to the ground truth (blue line), and are well within the confidence intervals of our forecast.

In [ ]:
# Plot the dynamic forecast with confidence intervals.

ax = data_diff['1990':].plot(label='observed', figsize=(20, 15))
pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=ax)

ax.fill_between(pred_dynamic_conf.index,
                pred_dynamic_conf.iloc[:, 0],
                pred_dynamic_conf.iloc[:, 1], color='g', alpha=.3)

ax.fill_betweenx(ax.get_ylim(), pd.to_datetime('1996-05-01'), chicago_forecasted.index[-1], alpha=.1, zorder=-1)

ax.set_xlabel('Date')
ax.set_ylabel('Value')

plt.legend()
plt.show()

Once again, we quantify the predictive performance of our forecasts by computing the MSE.

In [ ]:
# Extract the predicted and true values of our time series
chicago_forecasted = pred_dynamic.predicted_mean
chicago_truth = data_diff['1996-05-01':]

# Compute the mean square error
mse = ((chicago_forecasted - chicago_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

The predicted values obtained from the dynamic forecasts yield an MSE of 1.01. This is slightly higher than the one-step ahead, which is to be expected given that we are relying on less historical data from the time series.

Both the one-step ahead and dynamic forecasts confirm that this time series model is valid. However, much of the interest around time series forecasting is the ability to forecast future values way ahead in time.

### Producing and Visualizing Forecasts

We will now describe how to leverage our seasonal ARIMA time series model to forecast future values. The .get_forecast() method of our time series output can compute forecasted values for a specified number of steps ahead.

In [ ]:
# Get forecast 500 steps ahead in future
prediction = output.get_forecast(steps=500)

# Get confidence intervals of forecasts
pred_conf = prediction.conf_int()

We can use the output of this code to plot the time series and forecasts of its future values.

In [ ]:
# Plot future predictions with confidence intervals
ax = data_diff.plot(label='observed', figsize=(20, 15))
prediction.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='k', alpha=0.25)
ax.set_xlabel('Date')
ax.set_ylabel('Value')

plt.legend()
plt.show()

As we forecast further out into the future, it is natural for us to become less confident in our values. This is reflected by the confidence intervals generated by our model, which grow larger as we move further out into the future.